In [49]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import numpy as np
from sqlalchemy import types, create_engine
from sqlalchemy.engine.url import URL
import psycopg2
from sqlalchemy.orm import sessionmaker
import pyodbc

In [50]:
user = 'postgres'
server = 'localhost'
database = 'postgres'

In [51]:
def to_postgres(user,server,database):
    engine_postgres = create_engine(f'postgresql+psycopg2://postgres:{user}@{server}/{database}')
    return engine_postgres

In [52]:
engine = to_postgres(user,server,database)
df = pd.read_sql('nasabah',con=engine)

In [53]:
import pandas as pd
import lightgbm as lgb

# Drop unnecessary columns
# Transform 'tanggal_data' to numerical features
forecast = df
forecast['tanggal_data'] = pd.to_datetime(forecast['tanggal_data'])
forecast['year'] = forecast['tanggal_data'].dt.year
forecast['month'] = forecast['tanggal_data'].dt.month
forecast['day'] = forecast['tanggal_data'].dt.day

# Create lag features
lag_features = ['usia_hari', 'tunggak_pokok', 'tunggak_bunga', 'frekuensi_tunggakan', 'suku_bunga_acuan']

for feature in lag_features:
    for lag in range(1, 4):  # Lag hingga 3 bulan
        forecast[f'{feature}_lag{lag}'] = forecast.groupby('nomor_rekening_loan')[feature].shift(lag)

columns_to_drop = ['norek', 'tanggal_buka', 'tanggal_tutup', 'tipe_segment','kolek_nasabah_encoded','tanggal_data','nomor_rekening_loan']
forecast = forecast.drop(columns=columns_to_drop)
forecast['cif_no'] = forecast['cif_no'].astype('int')
forecast = forecast.fillna(0)

# Function to split data
def split_data(forecast):
    # Define the last month and threshold date
    last_year_month = forecast[['year', 'month']].max()
    last_month_start = pd.Timestamp(year=last_year_month['year'], month=last_year_month['month'], day=1)

    # Split the data
    train_data = forecast[(forecast['year'] < last_month_start.year) | 
                          ((forecast['year'] == last_month_start.year) & (forecast['month'] < last_month_start.month))]
    test_data = forecast[(forecast['year'] == last_month_start.year) & (forecast['month'] == last_month_start.month)]

    return train_data, test_data

# Split the data
train_data, test_data = split_data(forecast)

# Define features and target
target = 'kolek_nasabah'

X_train = train_data.drop(columns=target)
y_train = train_data[target]
X_test = test_data.drop(columns=target)
y_test = test_data[target]

# Train the LightGBM model
model = lgb.LGBMClassifier(random_state=42, class_weight='balanced')
model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000684 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4194
[LightGBM] [Info] Number of data points in the train set: 6706, number of used features: 20
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438


LGBMClassifier(class_weight='balanced', random_state=42)

In [54]:
X_train

,cif_no,saldo_psak,plafond,usia_hari,usia_jatuh_tempo,tunggak_pokok,tunggak_bunga,denda,frekuensi_tunggakan,angsuran,...,tunggak_pokok_lag3,tunggak_bunga_lag1,tunggak_bunga_lag2,tunggak_bunga_lag3,frekuensi_tunggakan_lag1,frekuensi_tunggakan_lag2,frekuensi_tunggakan_lag3,suku_bunga_acuan_lag1,suku_bunga_acuan_lag2,suku_bunga_acuan_lag3
0,207797,1.761031e+08,2.303540e+08,1733,857.63,130323.34,981095.02,3655.33,8.6,1779341.47,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,346222,2.732027e+08,3.800159e+08,3440,50.40,860870.28,118394.86,3575.41,8.7,7823840.12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,305089,1.989460e+07,2.269266e+07,4168,1490.10,952295.76,729967.85,4468.09,5.7,1968549.10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,624727,6.698834e+08,7.137429e+08,1888,99.35,275043.47,517184.72,3784.29,1.4,3876061.56,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,651949,6.867347e+08,9.137683e+08,4767,1047.03,385667.65,789012.75,425.57,0.1,1556118.71,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9993,726895,2.562378e+08,3.142231e+08,2133,1157.39,761372.36,324442.82,2059.79,5.0,7228274.98,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9995,551035,1.748207e+08,1.761287e+08,1350,338.51,215994.50,90615.72,1746.20,8.9,3264209.36,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9996,140020,6.663322e+08,9.809310e+08,2302,1736.80,140573.92,102351.88,3808.84,5.2,8204432.17,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9997,581366,5.574637e+08,8.016715e+08,543,1641.92,481636.51,395925.63,2985.20,1.6,474035.73,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
#export model
import pickle

# Save the model to a pickle file
with open('lgb_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [56]:
df = pd.read_csv('sample_data_test_2024.csv')
# Transform 'tanggal_data' to numerical features
df['tanggal_data'] = pd.to_datetime( df['tanggal_data'])
df['year'] =  df['tanggal_data'].dt.year
df['month'] =  df['tanggal_data'].dt.month
df['day'] =  df['tanggal_data'].dt.day

# Sort and reset index
df =  df.sort_values(by=['nomor_rekening_loan', 'year', 'month', 'day']).reset_index(drop=True)

# Create lag features
lag_features = ['usia_hari', 'tunggak_pokok', 'tunggak_bunga', 'frekuensi_tunggakan', 'suku_bunga_acuan']

for feature in lag_features:
    for lag in range(1, 4):  # Lag hingga 3 bulan
         df[f'{feature}_lag{lag}'] =  df.groupby('nomor_rekening_loan')[feature].shift(lag)

df =  df.fillna(0)

In [57]:
from datetime import datetime, timedelta
def start_of_next_month(df):
    # Get the current date
    today = max(df['tanggal_data'])

    # Calculate the first day of the next month
    if today.month == 12:
        next_month = datetime(today.year + 1, 1, 1)
    else:
        next_month = datetime(today.year, today.month + 1, 1)

    return next_month

In [58]:
pred_date = start_of_next_month(df)

In [59]:
pred_date

datetime.datetime(2025, 1, 1, 0, 0)

In [60]:
data_pred = df[df['tanggal_data'] == max(df['tanggal_data'])]
columns_to_drop = ['norek', 'tanggal_buka', 'tanggal_tutup', 'tipe_segment','kolek_nasabah_encoded','tanggal_data','nomor_rekening_loan','kolek_nasabah']
data_pred.drop(columns=columns_to_drop, inplace=True)
data_pred

C:\Users\Hanif\AppData\Local\Temp\ipykernel_33064\2196564614.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_pred.drop(columns=columns_to_drop, inplace=True)


,cif_no,saldo_psak,plafond,usia_hari,usia_jatuh_tempo,tunggak_pokok,tunggak_bunga,denda,frekuensi_tunggakan,angsuran,...,tunggak_pokok_lag3,tunggak_bunga_lag1,tunggak_bunga_lag2,tunggak_bunga_lag3,frekuensi_tunggakan_lag1,frekuensi_tunggakan_lag2,frekuensi_tunggakan_lag3,suku_bunga_acuan_lag1,suku_bunga_acuan_lag2,suku_bunga_acuan_lag3
12,130918415124,4.070651e+08,5.236520e+08,1638,1437.076760,999508.19,180617.34,1.09,7.9,9185242.17,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,700145461967,6.867217e+08,7.263855e+08,1264,544.414443,403428.90,305035.67,1996.52,0.6,8093301.23,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193,542842000382,5.887818e+08,8.014491e+08,620,403.154502,273297.84,95230.82,524.95,8.4,3677062.07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
223,370865495850,4.998819e+08,5.436414e+08,1567,798.474745,981918.20,462943.12,2775.59,7.7,7700284.22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
247,657003901596,2.589585e+08,3.848591e+08,1977,1795.858279,895366.39,552459.94,4695.59,8.8,3260420.51,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9736,576269251655,3.081991e+08,4.085078e+08,4940,1936.923041,245072.66,666445.82,4583.92,9.8,7074947.83,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9781,929191310904,4.119538e+08,5.976716e+08,860,455.900501,50373.95,550257.49,363.18,7.4,2685374.29,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9807,772837718900,4.195531e+08,5.061226e+08,4487,1136.833454,399621.78,438917.96,3769.05,4.8,3073500.53,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9898,861525218664,2.893381e+08,3.099637e+08,1313,1856.803367,778484.51,206168.54,1640.11,2.1,8602077.60,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
